## Import the required libraries

Reference: https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f

In [12]:
import numpy as np
import pandas as pd

In [13]:
data = pd.read_csv('Customer_Behaviour.csv')
data

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


Clarity on entropy and Info gain: https://www.youtube.com/watch?v=9r7FIXEAGvs

In [14]:
data.drop('User ID', axis = 1, inplace = True)

data['Age_Label'] = pd.cut(x=data['Age'], bins=[17, 30, 45, 60], 
                     labels=['Young adults', 'Middle Age', 'Seniors'])
data['Salary_Label'] = pd.cut(x=data['EstimatedSalary'], bins=[14999, 60000, 105000, 150000], 
                     labels=['Low', 'Medium', 'High']) 

data_1 = data.pop('Purchased')
data['Purchased']=data_1

data.drop(['Age','EstimatedSalary'], axis = 1, inplace=True)
data_m = data.copy()                                                # Create copy of pandas DataFrame
data['Purchased'] = data_m['Purchased'].map({1: 'Yes', 0: 'No'})


data

,Gender,Age_Label,Salary_Label,Purchased
0,Male,Young adults,Low,No
1,Male,Middle Age,Low,No
2,Female,Young adults,Low,No
3,Female,Young adults,Low,No
4,Male,Young adults,Medium,No
...,...,...,...,...
395,Female,Seniors,Low,Yes
396,Male,Seniors,Low,Yes
397,Female,Seniors,Low,Yes
398,Male,Middle Age,Low,No


### The next main step is to convert numeric data into labels that can easily be categorized.

Ref: https://www.geeksforgeeks.org/pandas-cut-continuous-to-categorical/

#### Upon analysis, the ages range from 18-60, hence we categorize them as 18-30 as 'young adults', 31-45 as 'middle age' and 46-60 as 'seniors'

#### Upon analysis, the salaries range from 15000-150000, hence we categorize them as: 15000-60000 as 'low', 60001-105000 as 'medium' and 105001-150000 as 'high'

## Rearranging for convenience

## How is entropy calculated?

Total row = 400

Row with "1" class = 250 (assume)

Row with "0" class = 150 (assume)

Complete entropy of dataset is -

H(S) = - p(1) * log2(p(1)) - p(0) * log2(p(0))

     = - (250/400) * log2(250/400) - (150/400) * log2(150/400)
     
     = (some value)

In [15]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset - 400
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr

## Calculating the entropy for the filtered dataset

Categorical values of Gender - male and female

Total males = 300 (assume)

Male and 1 = 100

Male and 0 = 200

H(Gender=Male) = -(100/300)*log(100/300)-(200/300)*log(200/300) = (some value)

etc. for female

Repeat the process for Age_label and Salary_Label as well

In [16]:
def calc_entropy(feature_value_data, label, class_list): #f_v_d - Gender = male
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

## Calculating information gain for a feature

I(Gender) = p(Male) * H(Gender=Male) + p(Female) * H(Gender=Female) 

= (300/400)*H(male) + (100/400)*H(female) = some value

Information Gain = H(S) - I(Outlook)

                 = 0.94 - 0.693 (assume)
                 
                 = 0.247 (assume)

In [17]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unique values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

## Finding the most informative feature

Information gain: (assume)

  Gender = 0.247 (Highest value)
  
  Age = 0.0292
  
  EstimatedSalary = 0.153

In [18]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

## Adding a node to the tree

As we have found the feature name with the highest information gain, we have to generate a node in the tree and its value as a branch. For example, we have selected Gender, so we have to add Gender as a node in the tree and its value Male or Female as a branch.

In [19]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further, 
                                      #so the branch is marking with ?
            
    return tree, train_data

## Performing ID3 Algorithm and generating Tree

Now, we should ensemble the methods which should recursively do Step 4 — Step 8.

In [20]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

## Finding unique classes of the label and Starting the algorithm:

We can start calling the recursive tree building algorithm of ID3 after finding the class names.

In [21]:
data

,Gender,Age_Label,Salary_Label,Purchased
0,Male,Young adults,Low,No
1,Male,Middle Age,Low,No
2,Female,Young adults,Low,No
3,Female,Young adults,Low,No
4,Male,Young adults,Medium,No
...,...,...,...,...
395,Female,Seniors,Low,Yes
396,Male,Seniors,Low,Yes
397,Female,Seniors,Low,Yes
398,Male,Middle Age,Low,No


In [22]:
def id3(data_1, label):
    train_data = data.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

In [ ]:
tree = id3(data_1, 'Purchased')

In [ ]:
tree